# Regression-1: Build quick models

ここではモデル構築の大きな流れを掴みましょう。<br>まずは、サンプルデータ（回帰用）を読み込みます。

In [ ]:
# import the data for regression
import pandas as pd
from IPython.core.display import display
from sklearn.datasets import load_boston
dataset = load_boston()

# set dataframe
X = pd.DataFrame(dataset.data,columns=dataset.feature_names)
y = pd.Series(dataset.target,name='y')

# check the shape
print('--------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('--------------------------------------')
print(y.describe())
display(X.join(y).head())

アンサンブル学習（ブースティング）の一つである勾配ブースティンレグレッサーを読み込み学習（fit）します。予測値と正解データの整合性をR2値で評価しています。データを読み込み、学習させ、評価する流れを掴んで下さい。

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# build a model
est = GradientBoostingRegressor(n_estimators=100,
                                max_depth=3,
                                random_state=0)
est.fit(X,y)

# evaluate a model by R2 score
from sklearn.metrics import r2_score
y_pred = est.predict(X)
r2 = r2_score(y, y_pred)
print('R2 score: %.3f' % r2)

モデル構築は以上ですが、事前の精度確認作業を含む場合、モデル構築ステップは増加します。それは未知データへの当てはまりを確認する必要があるためです。予測上の最大の敵は<b>過学習</b>であり、この過学習へ対処するためのステップが増えると理解して下さい。<b>ホールドアウト法</b>はその対処法の一つで、汎化性能を確認しベストなモデルを選択するのに役立ちます。ここではモデル比較のため、パラメータ違いの勾配ブースティングモデルを準備します。

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# ホールドアウト
# 訓練を80%,テストを20%に分割
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.20,
                                                 random_state=1)
# 二つのモデルを構築(比較用)
# Standard model
est1 = GradientBoostingRegressor(max_depth=3,random_state=1)
est1.fit(X_train,y_train)
# Complex model
est2 = GradientBoostingRegressor(max_depth=10,random_state=1) 
est2.fit(X_train,y_train)

# モデルパフォーマンス指標(R2)の取得
# for training data
r2_est1_train = r2_score(y_train,est1.predict(X_train))
r2_est2_train = r2_score(y_train,est2.predict(X_train))
# for test data
r2_est1_test = r2_score(y_test,est1.predict(X_test))
r2_est2_test = r2_score(y_test,est2.predict(X_test))

# 性能指標の表示
print('Train Score(est1,est2):(%.3f,%.3f)'% (r2_est1_train, r2_est2_train))
print('Test Score(est1,est2):(%.3f,%.3f)'% (r2_est1_test, r2_est2_test))